#获取近10年股票日线数据

In [1]:
import tushare as ts
token='829a1fbce8eb0e34f05ab19906d0e08227c6f64261a81272aa078ccd'
pro=ts.pro_api(token)
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from datetime import datetime, timedelta
import numpy as np

In [2]:
# 获取近10年上市公司的数据
df_stock = pro.stock_basic(exchange='', list_status='L', fields='ts_code,symbol,name,area,industry,list_date')
df_stock.head()

,ts_code,symbol,name,area,industry,list_date
0,000001.SZ,000001,平安银行,深圳,银行,19910403
1,000002.SZ,000002,万科A,深圳,全国地产,19910129
2,000004.SZ,000004,ST国华,深圳,软件服务,19910114
3,000005.SZ,000005,ST星源,深圳,环境保护,19901210
4,000006.SZ,000006,深振业A,深圳,区域地产,19920427


In [3]:
df_stock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4940 entries, 0 to 4939
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ts_code    4940 non-null   object
 1   symbol     4940 non-null   object
 2   name       4940 non-null   object
 3   area       4826 non-null   object
 4   industry   4826 non-null   object
 5   list_date  4940 non-null   object
dtypes: object(6)
memory usage: 231.7+ KB


In [4]:
df_stock.describe()

,ts_code,symbol,name,area,industry,list_date
count,4940,4940,4940,4826,4826,4940
unique,4940,4940,4939,32,110,2477
top,000001.SZ,000001,三维股份,浙江,软件服务,20200727
freq,1,1,2,625,267,31


In [5]:
df_stock['list_date']

0       19910403
1       19910129
2       19910114
3       19901210
4       19920427
          ...   
4935    20211025
4936    20220927
4937    20220415
4938    20220609
4939    20201029
Name: list_date, Length: 4940, dtype: object

In [6]:
# convert list_date to datetime
df_stock['list_date'] = pd.to_datetime(df_stock['list_date'], format='%Y%m%d')
df_stock['list_date']

0      1991-04-03
1      1991-01-29
2      1991-01-14
3      1990-12-10
4      1992-04-27
          ...    
4935   2021-10-25
4936   2022-09-27
4937   2022-04-15
4938   2022-06-09
4939   2020-10-29
Name: list_date, Length: 4940, dtype: datetime64[ns]

In [7]:
# 筛选出2012年以前上市的公司
df_stock = df_stock[df_stock['list_date'] < '2012-01-01']
df_stock.describe()

/var/folders/fs/n2w57x6j4rx87rzzzxxf_z4m0000gn/T/ipykernel_8718/3386739458.py:3: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df_stock.describe()


,ts_code,symbol,name,area,industry,list_date
count,2215,2215,2215,2215,2215,2215
unique,2215,2215,2215,32,110,1195
top,000001.SZ,000001,平安银行,浙江,电气设备,2009-10-30 00:00:00
freq,1,1,1,226,115,26
first,NaN,NaN,NaN,NaN,NaN,1990-12-10 00:00:00
last,NaN,NaN,NaN,NaN,NaN,2011-12-29 00:00:00


In [28]:
stock_pool = df_stock['ts_code'].tolist()
# sb Tushare单次最多取6000条数据 
stock_10yr = pd.DataFrame()
for i in range(len(stock_pool)):
    stock_10yr_tmp = pro.daily(ts_code=stock_pool[i], start_date='20120101', end_date='20220928')
    stock_10yr = pd.concat([stock_10yr, stock_10yr_tmp], ignore_index=True)
stock_10yr.to_csv('Astock_10yr.csv', index=False)

In [29]:
stock_10yr.head()

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,000001.SZ,20220928,12.09,12.25,11.92,12.11,12.15,-0.04,-0.3292,780925.58,946211.767
1,000001.SZ,20220927,12.00,12.16,11.81,12.15,12.00,0.15,1.2500,789622.29,946732.994
2,000001.SZ,20220926,12.16,12.28,11.99,12.00,12.29,-0.29,-2.3596,903729.04,1094060.839
3,000001.SZ,20220923,12.24,12.43,12.23,12.29,12.29,0.00,0.0000,586441.06,722703.128
4,000001.SZ,20220922,12.33,12.36,12.25,12.29,12.43,-0.14,-1.1263,586133.38,720058.210


In [30]:
stock_10yr.describe()

,open,high,low,close,pre_close,change,pct_chg,vol,amount
count,5.507443e+06,5.507443e+06,5.507443e+06,5.507443e+06,5.507443e+06,5.507443e+06,5.507443e+06,5.507443e+06,5.507443e+06
mean,1.311142e+01,1.339824e+01,1.285704e+01,1.313259e+01,1.312653e+01,6.062872e-03,6.336908e-02,1.799348e+05,2.001737e+05
std,2.610682e+01,2.656177e+01,2.570212e+01,2.614301e+01,2.612143e+01,7.919416e-01,3.034715e+00,4.354825e+05,4.991437e+05
min,6.700000e-01,6.700000e-01,6.700000e-01,6.700000e-01,6.700000e-01,-1.719800e+02,-7.245000e+01,7.000000e+00,6.202000e+00
25%,5.800000e+00,5.910000e+00,5.700000e+00,5.810000e+00,5.810000e+00,-1.200000e-01,-1.358900e+00,3.307389e+04,2.821399e+04
50%,9.100000e+00,9.270000e+00,8.940000e+00,9.110000e+00,9.110000e+00,0.000000e+00,0.000000e+00,7.582207e+04,7.064183e+04
75%,1.500000e+01,1.532000e+01,1.471000e+01,1.502000e+01,1.502000e+01,1.200000e-01,1.371300e+00,1.783719e+05,1.848713e+05
max,2.587980e+03,2.627880e+03,2.485000e+03,2.601000e+03,2.601000e+03,1.610000e+02,9.860700e+02,5.135467e+07,6.794128e+07


In [31]:
# get name from ts_code
def get_name(ts_code):
    return df_stock[df_stock['ts_code'] == ts_code]['name'].values[0]

In [32]:
get_name('000001.SZ')

'平安银行'